Summon libraries.

In [95]:
# Summon libraries.
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel as OrdModel
import pandas as pd
from pandas import Series as ser
from pandas import Categorical as cat
from scipy import stats
import numpy as np

Call and work data.

In [96]:
# Call data. Uses PANDAS.
admissions = pd.read_csv('.\data\math dept.csv')

In [97]:
# Set admission categories.
admissions['Status'] = pd.Categorical(admissions.Status)
admissions['Status'] = admissions.Status.cat.reorder_categories(['Enrolled','Admitted','Applied'], ordered = True)
# Merge two or more races, unknown, and visas.
mapping = {'Two or More Races' : 'Other', 'Unknown' : 'Other', 'Visa Non-U.S.' : 'Other'}
admissions['Ethnicity'] = admissions['Ethnicity'].replace(mapping)
# Discard unknown genders.
admissions = admissions[admissions['Sex'] != 'Unknown']
# Since each model has identical formula, develop it once.
formula = 'Status ~ HS_GPA + C(Major,Treatment("Pre-Applied Statistics")) + C(Ethnicity,Treatment("White")) + C(Sex,Treatment("Female")) + C(Local,Treatment("Non-Local"))'

## Logit

Logit models.

In [98]:
# Create formula, then create the model and output the model.
fitted_logit_f = OrdModel.from_formula(formula, admissions, distr = "logit")
fitted_logit = fitted_logit_f.fit(method='bfgs', disp=False)
print(fitted_logit.summary())

# Untransform models.
num_of_thresholds = 2
print(f"The actual thresholds are {fitted_logit_f.transform_threshold_params(fitted_logit.params[-num_of_thresholds:])}")

                             OrderedModel Results                             
Dep. Variable:                 Status   Log-Likelihood:                -515.71
Model:                   OrderedModel   AIC:                             1055.
Method:            Maximum Likelihood   BIC:                             1110.
Date:                Sun, 03 Dec 2023                                         
Time:                        02:06:04                                         
No. Observations:                 719                                         
Df Residuals:                     707                                         
Df Model:                          10                                         
                                                                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------
C(Major, Treatmen

In [99]:
# The three models don't vary in how many parameters they produce, 
# Thus, we can set model_param to count the number of parameters by fitted_logit.params.
# Pull df_model for degrees of freedom.
model_param = (fitted_logit.params).count()
deg_free = fitted_logit.df_model

Logit AIC, BIC, AICC.

In [100]:
# AIC, BIC are built into the regression. 
# AICC needs to be developed.
aic_logit = fitted_logit.aic
aicc_logit = sm.tools.eval_measures.aicc(
# Value of log likelihood function.
    fitted_logit.llf,
# Number of observations.
    fitted_logit.nobs,
    model_param)
bic_logit = fitted_logit.bic

Logit log likelihood.

In [101]:
# Pull null log likelihood function from initial model.
nullloglike_logit = (fitted_logit.llnull)

In [102]:
# Uses null and fitted log likelihoods to perform the deviance test.
deviance= -2 * (fitted_logit.llnull-(fitted_logit.llf))
print(f"Deviance statistic is {deviance}.")
# Chi2.cdf is from scipy.stats.
from scipy.stats import chi2
pvalue = 1 - chi2.cdf(deviance,deg_free)
print(f"p-value is {pvalue}.")

Deviance statistic is 319.51611773949.
p-value is 0.0.


Logit prediction.

In [103]:
# Prediction.
predict_val = pd.DataFrame(
    {"HS_GPA" : 3.54, "Major" : "Pre-Math Education", "Ethnicity" :  "White", "Sex": "Female", "Local": "Non-Local"}, index=[0])
predict_val = sm.add_constant(predict_val)

# Isolated. This one grabs the item "values" from the array.
print(f'Predicted: {fitted_logit.predict(predict_val).values[0]}')

Predicted: [0.06101266 0.66304283 0.27594451]


## Probit

Probit models.

In [104]:
# Create formula, then create the model and output the model.
fitted_probit_f = OrdModel.from_formula(formula, admissions, distr = "probit")
fitted_probit = fitted_probit_f.fit(method='bfgs', disp=False)
print(fitted_probit.summary())

# Untransform models.
num_of_thresholds = 2
print(f"The actual thresholds are {fitted_probit_f.transform_threshold_params(fitted_probit.params[-num_of_thresholds:])}")

                             OrderedModel Results                             
Dep. Variable:                 Status   Log-Likelihood:                -530.67
Model:                   OrderedModel   AIC:                             1085.
Method:            Maximum Likelihood   BIC:                             1140.
Date:                Sun, 03 Dec 2023                                         
Time:                        02:06:04                                         
No. Observations:                 719                                         
Df Residuals:                     707                                         
Df Model:                          10                                         
                                                                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------
C(Major, Treatmen

Probit AIC, BIC, AICC.

In [105]:
# AIC, BIC are built into the regression. 
# AICC needs to be developed.
aic_probit = fitted_probit.aic
aicc_probit = sm.tools.eval_measures.aicc(
# Value of log likelihood function.
    fitted_probit.llf,
# Number of observations.
    fitted_probit.nobs,
    model_param)
bic_probit = fitted_probit.bic

Probit log likelihood.

In [106]:
# Pull null log likelihood function from initial model.
nullloglike_probit = (fitted_probit.llnull)

In [107]:
# Uses null and fitted log likelihoods to perform the deviance test.
deviance= -2 * (fitted_probit.llnull-(fitted_probit.llf))
print(f"Deviance statistic is {deviance}.")
# Chi2.cdf is from scipy.stats.
from scipy.stats import chi2
pvalue = 1 - chi2.cdf(deviance,deg_free)
print(f"p-value is {pvalue}.")

Deviance statistic is 289.58490106782347.
p-value is 0.0.


Probit prediction.

In [108]:
# Prediction.
predict_val = pd.DataFrame(
    {"HS_GPA" : 3.54, "Major" : "Pre-Math Education", "Ethnicity" :  "White", "Sex": "Female", "Local": "Non-Local"}, index=[0])
predict_val = sm.add_constant(predict_val)

# Isolated. This one grabs the item "values" from the array.
print(f'Predicted: {fitted_probit.predict(predict_val).values[0]}')

Predicted: [0.08377838 0.66089276 0.25532886]


## Cloglog

Cloglog models.

In [109]:
# Develop cloglog.
class cloglog(stats.rv_continuous):
    def _ppf(self, q):
        return np.log(-np.log(1 - q))

    def _cdf(self, x):
        return 1 - np.exp(-np.exp(x))

cloglog = cloglog()

In [110]:
# Create formula, create the model, and output the model.
fitted_cloglog_f = OrdModel.from_formula(formula, admissions, distr = cloglog)
fitted_cloglog = fitted_cloglog_f.fit(method='bfgs', disp=False)
print(fitted_cloglog.summary())

# Untransform models.
num_of_thresholds = 2
print(f"The actual thresholds are {fitted_cloglog_f.transform_threshold_params(fitted_cloglog.params[-num_of_thresholds:])}")

                             OrderedModel Results                             
Dep. Variable:                 Status   Log-Likelihood:                -486.85
Model:                   OrderedModel   AIC:                             997.7
Method:            Maximum Likelihood   BIC:                             1053.
Date:                Sun, 03 Dec 2023                                         
Time:                        02:06:05                                         
No. Observations:                 719                                         
Df Residuals:                     707                                         
Df Model:                          10                                         
                                                                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------
C(Major, Treatmen

Cloglog AIC, BIC, AICC.

In [111]:
# AIC, BIC are built into the regression. 
# AICC needs to be developed.
aic_cloglog = fitted_cloglog.aic
aicc_cloglog = sm.tools.eval_measures.aicc(
# Value of log likelihood function.
    fitted_cloglog.llf,
# Number of observations.
    fitted_cloglog.nobs,
    model_param)
bic_cloglog = fitted_cloglog.bic

Cloglog log likelihood.

In [112]:
# Pull null log likelihood function from initial model.
nullloglike_cloglog = (fitted_cloglog.llnull)

In [113]:
# Uses null and fitted log likelihoods to perform the deviance test.
deviance= -2 * (fitted_cloglog.llnull-(fitted_cloglog.llf))
print(f"Deviance statistic is {deviance}.")
# Chi2.cdf is from scipy.stats.
from scipy.stats import chi2
pvalue = 1 - chi2.cdf(deviance,deg_free)
print(f"p-value is {pvalue}.")

Deviance statistic is 377.2318682908243.
p-value is 0.0.


Cloglog prediction.

In [114]:
# Prediction.
predict_val = pd.DataFrame(
    {"HS_GPA" : 3.54, "Major" : "Pre-Math Education", "Ethnicity" :  "White", "Sex": "Female", "Local": "Non-Local"}, index=[0])
predict_val = sm.add_constant(predict_val)

# Isolated. This one grabs the item "values" from the array.
print(f'Predicted: {fitted_cloglog.predict(predict_val).values[0]}')

Predicted: [0.02430106 0.31144394 0.664255  ]


## Model comparisons.

In [115]:
data = [[aic_logit,aicc_logit,bic_logit],
        [aic_logit,aicc_logit,bic_logit],
        [aic_cloglog,aicc_cloglog,bic_cloglog]]

comparison = pd.DataFrame(data,
                          columns = ['AIC','AICC','BIC'],
                          index = ['logit','logit','cloglog'])

comparison

,AIC,AICC,BIC
logit,1055.416336,1055.858263,1110.350673
logit,1055.416336,1055.858263,1110.350673
cloglog,997.700586,998.142512,1052.634922


Automated comparison.

In [117]:
# Initialize.
logit = 0
logit = 0
cloglog = 0 

# Runs through all 3 columns.
# If column has lowest value, that model +1 point.
# Model with highest points is declared at end.
def points(cur_min,logit,probit,cloglog):
        if cur_min == "logit":
                logit += 1
        elif cur_min == "logit":
                logit += 1
        else:
                cloglog += 1 
        return(logit,logit,cloglog)

# Runs through the models.
cur_min = comparison['AIC'].idxmin()
logit,logit,cloglog = points(cur_min,logit,logit,cloglog)
cur_min = comparison['AICC'].idxmin()
logit,logit,cloglog = points(cur_min,logit,logit,cloglog)
cur_min = comparison['BIC'].idxmin()
logit,logit,cloglog = points(cur_min,logit,logit,cloglog)

final = {logit:"logit",logit:"logit",cloglog:"cloglog"}

print(f"The model with the best fit is the cumulative {final.get(max(final))} model.")

The model with the best fit is the cumulative cloglog model.
